# Kidney segmentation from CT images 
This project is about employing convolutional neural network to solve a segmentation problem, the network is going to be feed with a set of images of CT kidney. First, it is necessary to do a pre-processing of the images in two step proccesses:
1. **Normalization of the intensities:** because the same regions of the kidneys can have very different intensities in different volumes.
2. **Truncate volume intensities:** in range of [-200,500] Housefield Units to remove irrelevant details. 
## Unet
Segmentation step is performed by U-Net architecture. 
![](/home/alejandrodf1/Documents/theory_project/teoria_project/references/unet.png) 

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
sys.path.insert(0,"../teoria_project")

In [27]:
import torch.nn.functional as F
import paths as path
from models import unet_parts
from models.unet_parts import *
#import teoria_project.utils.paths as path

In [28]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [3]:
#!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB ? eta 0:00:000:00:0100:08


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter